# Analysis of effects of sentiment of tweets on stock prices
In this project our aim is to find how sentiments of tweets on twitter effect the fluctuation of valuation of a company.

We have chosen 5 companies :-


1.   Apple
2.   Tesla
3.   Google
4.   Microsoft
5.   Amazon



### Importing Libraries

In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
!pip install varname
from varname import nameof
import os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Creating function for sentiment analysis
We have used the vader_lexicon as it works well for internet reviews and tweets as it takes into consideration emojis as well which are quite prevalent in communication methods over internet

In [91]:
# For sentiment analysis, we use VADER library
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

# Sentiment Analysis function
def sentiment_analysis(tweet):
    score = sid.polarity_scores(tweet)["compound"]
    return score

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### Importing data 
Converting data from csv files to panda data frame

In [92]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
# changing the directory to read the database

# reading the database
raw_company_db = pd.read_csv("/content/drive/MyDrive/Company.csv")
raw_company_tweet_db = pd.read_csv("/content/drive/MyDrive/Company_Tweet.csv")
raw_tweet_db = pd.read_csv("/content/drive/MyDrive/Tweet.csv")
raw_companyvalue_db = pd.read_csv("/content/drive/MyDrive/CompanyValues.csv")

### Viewing Current Data frames

In [94]:
raw_company_db.head(2)

,ticker_symbol,company_name
0,AAPL,apple
1,GOOG,Google Inc


In [95]:
raw_company_tweet_db.head(2)

,tweet_id,ticker_symbol
0,550803612197457920,AAPL
1,550803610825928706,AAPL


In [96]:
raw_tweet_db.head(2)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,550441509175443456,VisualStockRSRC,1420070457,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,1420070496,Insanity of today weirdo massive selling. $aap...,0,0,0


In [97]:
raw_companyvalue_db.head(2)

,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value
0,AAPL,2020-05-29,317.94,38399530,319.25,321.15,316.47
1,AAPL,2020-05-28,318.25,33449100,316.77,323.44,315.63


## Data Cleaning


### Merging the DataFrames to form one single DataFrame

In [98]:
# Part1: Merge raw_company_tweet_db, raw_company_db
raw_company_tweet_db = pd.merge(raw_company_tweet_db, raw_company_db,on='ticker_symbol')

# Part2: Merge raw_company_tweet_db, raw_company_db
raw_tweet_db = pd.merge(raw_tweet_db, raw_company_tweet_db, on="tweet_id")

In [99]:
# Veiwing the marged Dataframe 
raw_tweet_db.head(2)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,ticker_symbol,company_name
0,550441509175443456,VisualStockRSRC,1420070457,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,AAPL,apple
1,550441672312512512,KeralaGuy77,1420070496,Insanity of today weirdo massive selling. $aap...,0,0,0,AAPL,apple


In [100]:
# Saving the names of dataframes so we can use it if required in future
raw_tweet_db.name = nameof(raw_tweet_db)
raw_companyvalue_db.name = nameof(raw_companyvalue_db)

In [101]:
list_df = [raw_tweet_db, raw_companyvalue_db]

### Missing values check: Let us check if there are any missing values

In [102]:
[print(f"For DataFrame {df.name}, we have missing values check as\n{df.isna().sum()}\n\n") for df in list_df]

For DataFrame raw_tweet_db, we have missing values check as
tweet_id             0
writer           55919
post_date            0
body                 0
comment_num          0
retweet_num          0
like_num             0
ticker_symbol        0
company_name         0
dtype: int64


For DataFrame raw_companyvalue_db, we have missing values check as
ticker_symbol    0
day_date         0
close_value      0
volume           0
open_value       0
high_value       0
low_value        0
dtype: int64




[None, None]

In [103]:
raw_tweet_db.writer = raw_tweet_db.writer.fillna('anonymous')

In [104]:
# let us check if all null values are replaced
raw_tweet_db.isna().sum()

tweet_id         0
writer           0
post_date        0
body             0
comment_num      0
retweet_num      0
like_num         0
ticker_symbol    0
company_name     0
dtype: int64

## Data Engineering


We will convert:
1.   'post_date' column in raw_tweet_db
2.   'day_date' column in raw_companyvalue_db

to datetime for further processing

In [105]:
raw_tweet_db.post_date = pd.to_datetime(raw_tweet_db.post_date, unit="s")
raw_companyvalue_db.day_date = pd.to_datetime(raw_companyvalue_db.day_date)

In [106]:
raw_companyvalue_db.head(2)

,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value
0,AAPL,2020-05-29,317.94,38399530,319.25,321.15,316.47
1,AAPL,2020-05-28,318.25,33449100,316.77,323.44,315.63


On viewing the dataset we see that we have tweets available from 1 Jan 2015 abd the last tweet was on 31 Dec 2019 but the stocks are given till May 2020 so we'll drop those values whose corresponding tweets are not available

In [107]:
raw_companyvalue_db = raw_companyvalue_db[raw_companyvalue_db.day_date < "2020-01-01"]

In [108]:
raw_companyvalue_db.head()

,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value
103,AAPL,2019-12-31,293.65,25247630,289.93,293.68,289.5200
104,AAPL,2019-12-30,291.52,36059610,289.46,292.69,285.2200
105,AAPL,2019-12-27,289.80,36592940,291.12,293.97,288.1200
106,AAPL,2019-12-26,289.91,23334000,284.82,289.98,284.7000
107,AAPL,2019-12-24,284.27,12119710,284.69,284.89,282.9197


In order to check stock value performance rather than taking in absolute value we take the changes to analyze stocks better

In [109]:
# Let us add column to the stock price dataframe which shows the max stock price fluctuation
raw_companyvalue_db['fluctuation'] = raw_companyvalue_db.high_value - raw_companyvalue_db.low_value

# Let us add column to the stock price dataframe which shows the net rise in stock price
raw_companyvalue_db['price_gain'] = raw_companyvalue_db.close_value - raw_companyvalue_db.open_value

# Let us add column to the stock price dataframe which shows the total valuation at the end of the day
raw_companyvalue_db['total_valuation_EOD'] = raw_companyvalue_db.volume * raw_companyvalue_db.close_value

<ipython-input-109-69a1f3a09aa9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_companyvalue_db['fluctuation'] = raw_companyvalue_db.high_value - raw_companyvalue_db.low_value
<ipython-input-109-69a1f3a09aa9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_companyvalue_db['price_gain'] = raw_companyvalue_db.close_value - raw_companyvalue_db.open_value
<ipython-input-109-69a1f3a09aa9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [110]:
raw_companyvalue_db.head(3)

,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value,fluctuation,price_gain,total_valuation_EOD
103,AAPL,2019-12-31,293.65,25247630,289.93,293.68,289.52,4.16,3.72,7.413967e+09
104,AAPL,2019-12-30,291.52,36059610,289.46,292.69,285.22,7.47,2.06,1.051210e+10
105,AAPL,2019-12-27,289.80,36592940,291.12,293.97,288.12,5.85,-1.32,1.060463e+10


### Sentiment analysis on the tweets

In [111]:
raw_tweet_db['sentiment'] = raw_tweet_db['body'].apply(lambda x : sentiment_analysis(x))

In [112]:
raw_tweet_db.head()

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,ticker_symbol,company_name,sentiment
0,550441509175443456,VisualStockRSRC,2015-01-01 00:00:57,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,AAPL,apple,0.0000
1,550441672312512512,KeralaGuy77,2015-01-01 00:01:36,Insanity of today weirdo massive selling. $aap...,0,0,0,AAPL,apple,-0.8271
2,550441732014223360,DozenStocks,2015-01-01 00:01:50,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0,AMZN,Amazon.com,0.0000
3,550442977802207232,ShowDreamCar,2015-01-01 00:06:47,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1,TSLA,Tesla Inc,0.0000
4,550443807834402816,i_Know_First,2015-01-01 00:10:05,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL,apple,0.0000


In [113]:
# Considering there is a 'comment,  retweet & like' column, we can consider those tweets having the same sentiments
# So for counting the total number of tweets, we add a count column telling the trending score of the tweet which will be addition of all these 3 columns
# Adding 1 to trend score as tweet itself is one of the contributors to itself

raw_tweet_db.insert(7, "trend_score", raw_tweet_db.comment_num + raw_tweet_db.retweet_num + raw_tweet_db.like_num + 1)

Using an anchor to merge two data frames

In [114]:
# Creating Anchor Column for raw_tweet_db
raw_tweet_db.insert(3, "date_str", raw_tweet_db.post_date.astype("str").str.split(" "))
raw_tweet_db.date_str = [element[0] for element in raw_tweet_db.date_str]
raw_tweet_db.insert(0, "anchor", raw_tweet_db.date_str + raw_tweet_db.ticker_symbol)

# Creating Anchor Column for raw_companyvalue_db
raw_companyvalue_db.insert(
    2, "date_str", raw_companyvalue_db.day_date.astype("str").str.split(" ")
)
raw_companyvalue_db.date_str = [element[0] for element in raw_companyvalue_db.date_str]
raw_companyvalue_db.insert(
    0, "anchor", raw_companyvalue_db.date_str + raw_companyvalue_db.ticker_symbol
)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [115]:
# Merging the two dataframe
processed_db = pd.merge(raw_tweet_db, raw_companyvalue_db, on="anchor")

# Let us convert he string date column "date_str_x" to datetime
processed_db.date_str_x = pd.to_datetime(processed_db.date_str_x)

# Since we are aiming to see the impact of tweets on stock value (i.e. rise and fall), we can drop "neutral" sentiments
processed_db = processed_db[processed_db.sentiment != 0]

In [116]:
processed_db.head()

,anchor,tweet_id,writer,post_date,date_str_x,body,comment_num,retweet_num,like_num,trend_score,...,day_date,date_str_y,close_value,volume,open_value,high_value,low_value,fluctuation,price_gain,total_valuation_EOD
1,2015-01-01AAPL,550441672312512512,KeralaGuy77,2015-01-01 00:01:36,2015-01-01,Insanity of today weirdo massive selling. $aap...,0,0,0,1,...,2015-01-01,2015-01-01,110.38,41304780,112.82,113.13,110.21,2.92,-2.44,4.559222e+09
9,2015-01-01AAPL,550444112328261632,GetAOM,2015-01-01 00:11:17,2015-01-01,$UNP $ORCL $QCOM $MSFT $AAPL Top scoring mega ...,0,0,0,1,...,2015-01-01,2015-01-01,110.38,41304780,112.82,113.13,110.21,2.92,-2.44,4.559222e+09
13,2015-01-01AAPL,550445850170642432,JorelLaraKalel,2015-01-01 00:18:12,2015-01-01,@CNBC 15 Top #trades for #2015 #FastMoney $TIF...,0,0,2,3,...,2015-01-01,2015-01-01,110.38,41304780,112.82,113.13,110.21,2.92,-2.44,4.559222e+09
14,2015-01-01AAPL,550447574285418497,btcgemini,2015-01-01 00:25:03,2015-01-01,We searched through hundreds of charts and fou...,0,0,0,1,...,2015-01-01,2015-01-01,110.38,41304780,112.82,113.13,110.21,2.92,-2.44,4.559222e+09
15,2015-01-01AAPL,550447850857828352,JorelLaraKalel,2015-01-01 00:26:09,2015-01-01,Top 10 searched #stocks of #2014 $AAPL $FB $BA...,0,0,2,3,...,2015-01-01,2015-01-01,110.38,41304780,112.82,113.13,110.21,2.92,-2.44,4.559222e+09


### Making a clean data frame for further analysis

In [117]:
# Let us make a clean dataset with only the desired values
clean_db = processed_db[
    [
        "post_date",
        "date_str_x",
        "body",
        "trend_score",
        "ticker_symbol_x",
        "company_name",
        "sentiment",
        "close_value",
        "volume",
        "open_value",
        "high_value",
        "low_value",
        "fluctuation",
        "price_gain",
        "total_valuation_EOD"
    ]
]

In [118]:
# Let us check how many companies do we have in our dataset
print(
    f"In our dataset, we have total {len(clean_db.company_name.value_counts())} companies, namely\n{clean_db.company_name.value_counts()}"
)

In our dataset, we have total 5 companies, namely
apple         766291
Tesla Inc     713108
Amazon.com    426806
Google Inc    357476
Microsoft     188850
Name: company_name, dtype: int64


In [119]:
# after checking the ticker symbol, we see that Google Inc has 2 ticker symbols: GOOG and GOOGL
# Let us change that so all Google Inc tickers have the same symbol as GOOG
clean_db.ticker_symbol_x.replace("GOOGL", "GOOG", inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [120]:
# Let us make datasets for these 5 companies
apple_df = clean_db[clean_db.ticker_symbol_x == "AAPL"]
tesla_df = clean_db[clean_db.ticker_symbol_x == "TSLA"]
amazon_df = clean_db[clean_db.ticker_symbol_x == "AMZN"]
google_df = clean_db[clean_db.ticker_symbol_x == "GOOG"]
microsoft_df = clean_db[clean_db.ticker_symbol_x == "MSFT"]

In [121]:
trend_df = apple_df.groupby(by=["date_str_x","close_value"], as_index=False).agg({"trend_score":pd.Series.sum})
sentiment_df = apple_df.groupby(by=["date_str_x",], as_index=False).agg({"sentiment":pd.Series.mean})
final_apple = pd.merge(trend_df, sentiment_df, on='date_str_x')
final_apple.to_csv('final_apple.csv')
final_apple

,date_str_x,close_value,trend_score,sentiment
0,2015-01-01,110.38,1705,0.275451
1,2015-01-02,109.33,1539,0.169319
2,2015-01-03,109.33,330,0.392737
3,2015-01-04,109.33,801,0.370549
4,2015-01-05,106.25,1402,0.248010
...,...,...,...,...
1814,2019-12-27,289.80,1170,0.275813
1815,2019-12-28,289.80,520,0.300492
1816,2019-12-29,289.80,1045,0.315323
1817,2019-12-30,291.52,2761,0.333157


In [122]:
trend_df = google_df.groupby(by=["date_str_x","close_value"], as_index=False).agg({"trend_score":pd.Series.sum})
sentiment_df = google_df.groupby(by=["date_str_x",], as_index=False).agg({"sentiment":pd.Series.mean})
final_google = pd.merge(trend_df, sentiment_df, on='date_str_x')
final_google.to_csv('final_google.csv')
final_google

,date_str_x,close_value,trend_score,sentiment
0,2015-01-01,526.40,64,0.338186
1,2015-01-01,530.66,37,0.338186
2,2015-01-02,524.81,143,0.270801
3,2015-01-02,529.55,77,0.270801
4,2015-01-03,524.81,169,0.463743
...,...,...,...,...
3646,2019-12-29,1354.64,200,0.368337
3647,2019-12-30,1336.14,297,0.394844
3648,2019-12-30,1339.71,298,0.394844
3649,2019-12-31,1337.02,132,0.294916


In [123]:
trend_df = amazon_df.groupby(by=["date_str_x","close_value"], as_index=False).agg({"trend_score":pd.Series.sum})
sentiment_df = amazon_df.groupby(by=["date_str_x",], as_index=False).agg({"sentiment":pd.Series.mean})
final_amazon = pd.merge(trend_df, sentiment_df, on='date_str_x')
final_amazon.to_csv('final_amazon.csv')
final_amazon

,date_str_x,close_value,trend_score,sentiment
0,2015-01-01,310.35,1007,-0.221957
1,2015-01-02,308.52,534,0.184688
2,2015-01-03,308.52,61,0.259859
3,2015-01-04,308.52,90,0.287238
4,2015-01-05,302.19,242,0.318348
...,...,...,...,...
1821,2019-12-27,1869.80,3860,0.353117
1822,2019-12-28,1869.80,753,0.276010
1823,2019-12-29,1869.80,868,0.327622
1824,2019-12-30,1846.89,1498,0.355969


In [124]:
trend_df = tesla_df.groupby(by=["date_str_x","close_value"], as_index=False).agg({"trend_score":pd.Series.sum})
sentiment_df = tesla_df.groupby(by=["date_str_x",], as_index=False).agg({"sentiment":pd.Series.mean})
final_tesla = pd.merge(trend_df, sentiment_df, on='date_str_x')
final_tesla.to_csv('final_tesla.csv')
final_tesla

,date_str_x,close_value,trend_score,sentiment
0,2015-01-01,222.41,172,0.248615
1,2015-01-02,219.31,246,0.203091
2,2015-01-03,219.31,99,0.155294
3,2015-01-04,219.31,88,0.335715
4,2015-01-05,210.09,249,0.358320
...,...,...,...,...
1821,2019-12-27,430.38,13045,0.172203
1822,2019-12-28,430.38,5067,0.198432
1823,2019-12-29,430.38,5937,0.126686
1824,2019-12-30,414.70,13469,0.140709


In [125]:
trend_df = microsoft_df.groupby(by=["date_str_x","close_value"], as_index=False).agg({"trend_score":pd.Series.sum})
sentiment_df = microsoft_df.groupby(by=["date_str_x",], as_index=False).agg({"sentiment":pd.Series.mean})
final_microsoft = pd.merge(trend_df, sentiment_df, on='date_str_x')
final_microsoft.to_csv('final_microsoft.csv')
final_microsoft

,date_str_x,close_value,trend_score,sentiment
0,2015-01-01,46.450,81,0.204437
1,2015-01-02,46.760,118,0.265249
2,2015-01-03,46.760,58,0.219583
3,2015-01-04,46.760,30,0.304124
4,2015-01-05,46.325,172,0.240367
...,...,...,...,...
1821,2019-12-27,158.960,736,0.303387
1822,2019-12-28,158.960,249,0.257665
1823,2019-12-29,158.960,601,0.399279
1824,2019-12-30,157.590,508,0.407251
